In [1189]:
import pandas as pd
from datetime import date, timedelta
import urllib
import numpy as np
import altair as alt

In [1190]:
# Global variables
today = date.today()
yesterday = date.today() - timedelta(days=1)

In [1191]:
# Functions
def new_group(code, name, code_list, dataframe):
    df = dataframe[dataframe.code.isin(code_list)].groupby("date", as_index=False).sum()
    df["code"] = code
    df["name"] = name
    return df

def get_data(code,dataframe=our_utla):
    return dataframe[(dataframe["code"] == code) & ((dataframe["date"] == pd.to_datetime(yesterday)) | (dataframe["date"] == pd.to_datetime(today)))]

In [1192]:
# NHS
nhs_header_row = 15
nhs_trust_sheet_name = 'COVID19 total deaths by trust'
nhs_csv_trusts = 'nhs_csv_trusts.csv'
our_trust_codes = [
    'RYW', # Birmingham Community Healthcare NHS Foundation Trust
    'RNA', # The Dudley Group NHS Foundation Trust
    'RL4', # The Royal Wolverhampton NHS Trust
    'RXK', # Sandwell And West Birmingham Hospitals NHS Trust
    'RXW', # Shrewsbury And Telford Hospital NHS Trust
    'RRJ', # The Royal Orthopaedic Hospital NHS Foundation Trust
    'RRK', # University Hospitals Birmingham NHS Foundation Trust
    'RJE', # University Hospitals Of North Midlands NHS Trust
    'RBK', # Walsall Healthcare NHS Trust
    'RWP', # Worcestershire Acute Hospitals NHS Trust
    'R1A', # Worcestershire Health And Care NHS Trust
    'RL1', # The Robert Jones And Agnes Hunt Orthopaedic Hospital NHS Foundation Trust
]
nhs_file_format = "https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/DIRECTORY/COVID-19-total-announced-deaths-DATE.xlsx"
nhs_file_date_format = "%-d-%B-%Y"
nhs_file_directory_format = "%Y/%m"
nhs_latest_file = False

nhs_today_file = nhs_file_format.replace("DATE",today.strftime(nhs_file_date_format))
nhs_today_file = nhs_today_file.replace("DIRECTORY",today.strftime(nhs_file_directory_format))

nhs_yesterday_file = nhs_file_format.replace("DATE",yesterday.strftime(nhs_file_date_format))
nhs_yesterday_file = nhs_yesterday_file.replace("DIRECTORY",yesterday.strftime(nhs_file_directory_format))

## NHS file handling with fallback
try:
  urllib.request.urlopen(nhs_today_file)
  nhs_latest_file = nhs_today_file
except urllib.error.HTTPError as e:
  try:
    nhs_latest_file = urllib.request.urlopen(nhs_yesterday_file)
    nhs_latest_file = nhs_yesterday_file
  except urllib.error.HTTPError as e:
    pass

## Import a specific sheet from the NHS Excel file and convert to CSV
nhs_excel = pd.read_excel(nhs_latest_file, header=nhs_header_row, sheet_name=nhs_trust_sheet_name)
nhs_excel.to_csv(nhs_csv_trusts)
nhs_trusts = pd.read_csv(nhs_csv_trusts)

## Filtering down to our_trusts
our_trusts = nhs_trusts[nhs_trusts.Code.isin(our_trust_codes)] # filter to our trusts
our_trusts = our_trusts.filter(regex='(^Code$|^Name$|[0-9]{4}-.+|^Up to.01-Mar-20$)', axis=1) # filter to only the required columns
our_trusts = our_trusts.rename(columns={"Code": "code", "Name" : "name"}) # clean up column names
our_trusts = pd.melt(our_trusts, id_vars=["code","name"], var_name="date", value_name="deaths") # un-pivot the date columns
our_trusts["date"] = our_trusts["date"].str.replace("Up to 01-Mar-20","2020-02-29 00:00:00")
our_trusts["date"] = pd.to_datetime(our_trusts.date) # clean all dates to YYYY-MM-DD

## Adding cumulative deaths
our_trusts["cumulative_deaths"] = our_trusts.groupby("name").cumsum()

## Cleaning float
our_trusts["deaths"] = our_trusts["deaths"].convert_dtypes(convert_integer=True)
our_trusts["cumulative_deaths"] = our_trusts["cumulative_deaths"].convert_dtypes(convert_integer=True)

## Additional Groups / SubTotals
### Create Groups
#### Total Region
group_totalregion_trusts = new_group("ManualREG","Total Region",[
    'RYW', # Birmingham Community Healthcare NHS Foundation Trust
    'RNA', # The Dudley Group NHS Foundation Trust
    'RL4', # The Royal Wolverhampton NHS Trust
    'RXK', # Sandwell And West Birmingham Hospitals NHS Trust
    'RXW', # Shrewsbury And Telford Hospital NHS Trust
    'RRJ', # The Royal Orthopaedic Hospital NHS Foundation Trust
    'RRK', # University Hospitals Birmingham NHS Foundation Trust
    'RJE', # University Hospitals Of North Midlands NHS Trust
    'RBK', # Walsall Healthcare NHS Trust
    'RWP', # Worcestershire Acute Hospitals NHS Trust
    'R1A', # Worcestershire Health And Care NHS Trust
    'RL1', # The Robert Jones And Agnes Hunt Orthopaedic Hospital NHS Foundation Trust
],our_trusts)
#### Black Country and Birmingham
group_blackcountryandbirmingham_trusts = new_group("ManualBAB","Black Country and Birmingham",[
    'RYW', # Birmingham Community Healthcare NHS Foundation Trust
    'RNA', # The Dudley Group NHS Foundation Trust
    'RL4', # The Royal Wolverhampton NHS Trust
    'RXK', # Sandwell And West Birmingham Hospitals NHS Trust
    'RRJ', # The Royal Orthopaedic Hospital NHS Foundation Trust
    'RRK', # University Hospitals Birmingham NHS Foundation Trust
    'RBK', # Walsall Healthcare NHS Trust
],our_trusts)
#### Sandwell and Birmingham
group_sandwellandbirmingham_trusts = new_group("ManualSAB","Sandwell and Birmingham",[
    'RYW', # Birmingham Community Healthcare NHS Foundation Trust
    'RXK', # Sandwell And West Birmingham Hospitals NHS Trust
    'RRJ', # The Royal Orthopaedic Hospital NHS Foundation Trust
    'RRK', # University Hospitals Birmingham NHS Foundation Trust
],our_trusts)

#### Worcestershire
group_worcestershire_trusts = new_group("ManualWOR","Worcestershire",[
    'RWP', # Worcestershire Acute Hospitals NHS Trust
    'R1A', # Worcestershire Health And Care NHS Trust
],our_trusts)

### Add Groups
our_trust_groups = pd.DataFrame(columns = ["code","name","date","deaths","cumulative_deaths"])
our_trust_groups = our_trust_groups.append([
    group_sandwellandbirmingham_trusts,
    group_worcestershire_trusts,
    group_blackcountryandbirmingham_trusts,
    group_totalregion_trusts
], ignore_index=True)

In [1193]:
# Public Health England (PHE)
our_utla_codes = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E06000051', # Shropshire
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E06000020', # Telford and Wrekin
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
    'E10000034', # Worcestershire
]
pheh_dashboard_url = 'https://fingertips.phe.org.uk/documents/Historic%20COVID-19%20Dashboard%20Data.xlsx'

pheh_utla_header_row = 7 # zero indexed
pheh_utla_sheet = 'UTLAs'
pheh_utla_csv_filename = 'pheh-utla.csv'

pheh_ukdeaths_header_row = 7
pheh_ukdeaths_sheet = 'UK Deaths'
pheh_ukdeaths_csv_filename = 'pheh-ukdeaths.csv'

pheh_ukcases_header_row = 7
pheh_ukcases_sheet = 'Countries'
pheh_ukcases_csv_filename = 'pheh-ukcases.csv'

## Import the UK Deaths sheet from the PHE Excel file and convert to CSV
pheh_ukdeaths_excel = pd.read_excel(pheh_dashboard_url, header=pheh_ukdeaths_header_row, sheet_name=pheh_ukdeaths_sheet)
pheh_ukdeaths_excel.to_csv(pheh_ukdeaths_csv_filename)
pheh_ukdeaths = pd.read_csv(pheh_ukdeaths_csv_filename)

### Refining the UK Deaths data
pheh_ukdeaths = pheh_ukdeaths.rename(columns={
    "Date": "date",
    "Deaths" : "deaths",
}) # clean up column names
pheh_ukdeaths.drop('Unnamed: 0', axis='columns', inplace=True)
pheh_ukdeaths.drop('deaths', axis='columns', inplace=True)
pheh_ukdeaths = pd.melt(pheh_ukdeaths, id_vars=["date"], var_name="country", value_name="deaths") # un-pivot the date columns
pheh_ukdeaths["date"] = pd.to_datetime(pheh_ukdeaths.date) # clean all dates to YYYY-MM-DD
pheh_ukdeaths["deaths"] = pheh_ukdeaths["deaths"].convert_dtypes(convert_integer=True)

## Import the UTLA sheet from the PHE Excel file and convert to CSV
pheh_utla_excel = pd.read_excel(pheh_dashboard_url, header=pheh_utla_header_row, sheet_name=pheh_utla_sheet)
pheh_utla_excel.to_csv(pheh_utla_csv_filename)
pheh_utla = pd.read_csv(pheh_utla_csv_filename)

### Filtering down to our UTLAs
our_utla = pheh_utla.rename(columns={"Area Code": "code", "Area Name" : "name"}) # clean up column names
our_utla = our_utla[our_utla.code.isin(our_utla_codes)] # filter to our trusts
our_utla.drop('Unnamed: 0', axis='columns', inplace=True)
our_utla = pd.melt(our_utla, id_vars=["code","name"], var_name="date", value_name="cases") # un-pivot the date columns
our_utla["date"] = pd.to_datetime(our_utla.date) # clean all dates to YYYY-MM-DD
our_utla["diff"] = our_utla.groupby("code")["cases"].diff().convert_dtypes(convert_integer=True) # add daily variance

## Additional Groups / SubTotals
### Create Groups
#### Total Region
group_totalregion_cases = new_group("ManualREG","Total Region",[
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E06000051', # Shropshire
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E06000020', # Telford and Wrekin
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
    'E10000034', # Worcestershire
],our_utla)
#### Black Country
group_blackcountry_cases = new_group("ManualBLC","Black Country",[
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
],our_utla)
#### Black Country and Birmingham
group_blackcountryandbirmingham_cases = new_group("ManualBAB","Black Country and Birmingham",[
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
],our_utla)
#### Sandwell and Birmingham
group_sandwellandbirmingham_cases = new_group("ManualSAB","Sandwell and Birmingham",[
    'E08000025', # Birmingham
    'E08000028', # Sandwell
],our_utla)
#### Staffordshire
group_staffordshire_cases = new_group("ManualSTF","Staffordshire",[   
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
],our_utla)

### Add Groups
our_utla_groups = pd.DataFrame(columns = ["code","name","date","cases","diff"])
our_utla_groups = our_utla_groups.append([
    group_blackcountry_cases,
    group_sandwellandbirmingham_cases,
    group_staffordshire_cases,
    group_blackcountryandbirmingham_cases,
    group_totalregion_cases,
], ignore_index=True)

In [1194]:
## Data views
### Interesting
view_reductions = our_utla[our_utla["diff"] < 0] # instances where the cumulative case count went down
view_yesterday = our_utla[our_utla["date"] == pd.to_datetime(yesterday)]
view_today = our_utla[our_utla["date"] == pd.to_datetime(today)]
view_latest_deaths = our_trusts.pivot_table(our_trusts, index=["code","name"], aggfunc=np.sum)

### Required
#### Country
view_uk_cases = ""
view_uk_deaths = ""
view_england_cases = ""
view_england_deaths = ""
view_scotland_cases = ""
view_scotland_deaths = ""
view_wales_cases = ""
view_wales_deaths = ""
view_northernireland_cases = ""
view_northernireland_deaths = ""

#### Total Region
view_totalregion_cases = get_data("ManualREG",our_utla_groups)
view_totalregion_deaths = get_data("ManualREG",our_trust_groups)

#### Black Country and Birmingham
view_blackcountry_cases = get_data("ManualBLC",our_utla_groups)
view_blackcountryandbirmingham_cases = get_data("ManualBAB",our_utla_groups)
view_blackcountryandbirmingham_deaths = get_data("ManualBAB",our_trust_groups)

#### Wolverhampton
view_wolverhampton_cases = get_data("E08000031")
view_wolverhampton_deaths = get_data("RL4",our_trusts)

#### Walsall
view_walsall_cases = get_data("E08000030")
view_walsall_deaths = get_data("RBK",our_trusts)

#### Dudley
view_dudley_cases = get_data("E08000027")
view_dudley_deaths = get_data("RNA",our_trusts)

#### Sandwell and Birmingham
view_sandwell_cases = get_data("E08000028")
view_birmingham_cases = get_data("E08000025")
view_total_sandwellandbirmingham_cases = get_data("ManualSAB",our_utla_groups)
view_total_sandwellandbirmingham_deaths = get_data("ManualSAB",our_trust_groups)

#### Staffordshire
view_staffordshire_cases = get_data("E10000028")
view_stoke_cases = get_data("E06000021")
view_total_staffordshire_cases = get_data("ManualSTF",our_utla_groups)

#### Worcestershire
view_worcestershire_cases = get_data("E10000034")
view_worcetershire_deaths = get_data("ManualWOR",our_trust_groups)

In [1195]:
pheh_ukdeaths

,date,country,deaths
0,2020-03-10,UK,6
1,2020-03-11,UK,<NA>
2,2020-03-12,UK,8
3,2020-03-13,UK,<NA>
4,2020-03-14,UK,21
...,...,...,...
145,2020-04-04,Northern Ireland,48
146,2020-04-05,Northern Ireland,56
147,2020-04-06,Northern Ireland,63
148,2020-04-07,Northern Ireland,70


# UK

In [1196]:
view_uk_cases

''

In [1197]:
view_uk_deaths

''

# England

In [1198]:
view_england_cases

''

In [1199]:
view_england_deaths

''

# Scotland

In [1200]:
view_scotland_cases

''

In [1201]:
view_scotland_deaths

''

# Wales

In [1202]:
view_wales_cases

''

In [1203]:
view_wales_deaths

''

# Northern Ireland

In [1204]:
view_northernireland_cases

''

In [1205]:
view_northernireland_deaths

''

# Total Region
_Combining Black Country, Birmingham and Staffordshire._

In [1206]:
view_totalregion_cases

,code,name,date,cases,diff
154,ManualREG,Total Region,2020-04-08,4489,355


In [1207]:
view_totalregion_deaths

,code,name,date,deaths,cumulative_deaths
159,ManualREG,Total Region,2020-04-08,6,865


# Total Black Country and Birmingham

In [1208]:
view_blackcountry_cases

,code,name,date,cases,diff
30,ManualBLC,Black Country,2020-04-08,1515,122


In [1209]:
view_blackcountryandbirmingham_cases

,code,name,date,cases,diff
123,ManualBAB,Black Country and Birmingham,2020-04-08,2977,212


In [1210]:
view_blackcountryandbirmingham_deaths

,code,name,date,deaths,cumulative_deaths
119,ManualBAB,Black Country and Birmingham,2020-04-08,2,701


# Wolverhampton

In [1211]:
view_wolverhampton_cases

,code,name,date,cases,diff
307,E08000031,Wolverhampton,2020-04-08,354,19


In [1212]:
view_wolverhampton_deaths

,code,name,date,deaths,cumulative_deaths
474,RL4,THE ROYAL WOLVERHAMPTON NHS TRUST,2020-04-08,2,126


# Walsall

In [1213]:
view_walsall_cases

,code,name,date,cases,diff
306,E08000030,Walsall,2020-04-08,421,35


In [1214]:
view_walsall_deaths

,code,name,date,deaths,cumulative_deaths
477,RBK,WALSALL HEALTHCARE NHS TRUST,2020-04-08,0,37


# Dudley

In [1215]:
view_dudley_cases

,code,name,date,cases,diff
304,E08000027,Dudley,2020-04-08,341,32


In [1216]:
view_dudley_deaths

,code,name,date,deaths,cumulative_deaths
471,RNA,THE DUDLEY GROUP NHS FOUNDATION TRUST,2020-04-08,0,110


# Sandwell and Birmingham

In [1217]:
view_sandwell_cases

,code,name,date,cases,diff
305,E08000028,Sandwell,2020-04-08,399,36


In [1218]:
view_birmingham_cases

,code,name,date,cases,diff
303,E08000025,Birmingham,2020-04-08,1462,90


In [1219]:
view_total_sandwellandbirmingham_cases

,code,name,date,cases,diff
61,ManualSAB,Sandwell and Birmingham,2020-04-08,1861,126


In [1220]:
view_total_sandwellandbirmingham_deaths

,code,name,date,deaths,cumulative_deaths
39,ManualSAB,Sandwell and Birmingham,2020-04-08,0,428


# Staffordshire

In [1221]:
view_staffordshire_cases

,code,name,date,cases,diff
308,E10000028,Staffordshire,2020-04-08,673,76


In [1222]:
view_stoke_cases

,code,name,date,cases,diff
301,E06000021,Stoke-on-Trent,2020-04-08,130,13


In [1223]:
view_total_staffordshire_cases

,code,name,date,cases,diff
92,ManualSTF,Staffordshire,2020-04-08,803,89


# Worcestershire

In [1224]:
view_worcestershire_cases

,code,name,date,cases,diff
309,E10000034,Worcestershire,2020-04-08,439,20


In [1225]:
view_worcetershire_deaths

,code,name,date,deaths,cumulative_deaths
79,ManualWOR,Worcestershire,2020-04-08,1,56
